# Artigos 01 e 02 - descrevendo o modelo de negócios e investigando os dados crus

In [201]:
#importando bibliotecas
import numpy as np
import pandas as pd
import datetime as dt

In [202]:
#importando dados do csv para o dataframe
df = pd.read_csv('vendasBienio20222023Incognito.csv', sep = ";", low_memory=False)
df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Index           199177 non-null  int64 
 1   Date            199177 non-null  object
 2   Store ID        199177 non-null  int64 
 3   Category        199177 non-null  object
 4   Sale ID         199177 non-null  int64 
 5   Status          199177 non-null  object
 6   Payment Type    198324 non-null  object
 7   Place or Sale   199177 non-null  object
 8   Product         199177 non-null  object
 9   Product ID      199177 non-null  object
 10  Payment Method  71311 non-null   object
 11  Revenues        199177 non-null  object
 12  Products        199177 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 103.9 MB


In [203]:
# arquivo veio com duas colunas de índice, eliminando redundância:
df = df.drop(columns='Index')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Date            199177 non-null  object
 1   Store ID        199177 non-null  int64 
 2   Category        199177 non-null  object
 3   Sale ID         199177 non-null  int64 
 4   Status          199177 non-null  object
 5   Payment Type    198324 non-null  object
 6   Place or Sale   199177 non-null  object
 7   Product         199177 non-null  object
 8   Product ID      199177 non-null  object
 9   Payment Method  71311 non-null   object
 10  Revenues        199177 non-null  object
 11  Products        199177 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 102.4 MB


## Ajustando a tipagem das colunas

In [204]:
# ajustando a coluna date para tipo datetime:
df['Date'] = pd.to_datetime(df['Date'])

In [205]:
# ajustando a coluna revenues, removendo o cifrão e tipando como float
df['Revenues'] = df['Revenues'].apply(lambda x: x.replace('$',''))
df['Revenues'] = df['Revenues'].astype('float')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            199177 non-null  datetime64[ns]
 1   Store ID        199177 non-null  int64         
 2   Category        199177 non-null  object        
 3   Sale ID         199177 non-null  int64         
 4   Status          199177 non-null  object        
 5   Payment Type    198324 non-null  object        
 6   Place or Sale   199177 non-null  object        
 7   Product         199177 non-null  object        
 8   Product ID      199177 non-null  object        
 9   Payment Method  71311 non-null   object        
 10  Revenues        199177 non-null  float64       
 11  Products        199177 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 82.2 MB


In [206]:
#reduzindo o tamanho de armazenamento para colunas com números inteiros e floats
ints = df.select_dtypes(include=['int64','int32','int16']).columns
df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
floats = df.select_dtypes(include=['float']).columns
df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            199177 non-null  datetime64[ns]
 1   Store ID        199177 non-null  int16         
 2   Category        199177 non-null  object        
 3   Sale ID         199177 non-null  int32         
 4   Status          199177 non-null  object        
 5   Payment Type    198324 non-null  object        
 6   Place or Sale   199177 non-null  object        
 7   Product         199177 non-null  object        
 8   Product ID      199177 non-null  object        
 9   Payment Method  71311 non-null   object        
 10  Revenues        199177 non-null  float32       
 11  Products        199177 non-null  int8          
dtypes: datetime64[ns](1), float32(1), int16(1), int32(1), int8(1), object(7)
memory usage: 78.2 MB


## Usando o tipo de dados category

In [207]:
# usando dtype categórico para colunas com strings, a fim de economizar espaço - pois as colunas com string, nesse df, são atributos categóricos com dados nominais repetidos.
Objects = df.select_dtypes('object').columns
df[Objects] = df[Objects].apply(lambda x : x.astype('category'))
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            199177 non-null  datetime64[ns]
 1   Store ID        199177 non-null  int16         
 2   Category        199177 non-null  category      
 3   Sale ID         199177 non-null  int32         
 4   Status          199177 non-null  category      
 5   Payment Type    198324 non-null  category      
 6   Place or Sale   199177 non-null  category      
 7   Product         199177 non-null  category      
 8   Product ID      199177 non-null  category      
 9   Payment Method  71311 non-null   category      
 10  Revenues        199177 non-null  float32       
 11  Products        199177 non-null  int8          
dtypes: category(7), datetime64[ns](1), float32(1), int16(1), int32(1), int8(1)
memory usage: 5.4 MB


## Continuando com pequenos ajustes

In [208]:
# ajustando manualmente as colunas de Store ID e Sale ID
df['Store ID'] = df['Store ID'].astype('category')
df['Sale ID'] = df['Sale ID'].astype('category')
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            199177 non-null  datetime64[ns]
 1   Store ID        199177 non-null  category      
 2   Category        199177 non-null  category      
 3   Sale ID         199177 non-null  category      
 4   Status          199177 non-null  category      
 5   Payment Type    198324 non-null  category      
 6   Place or Sale   199177 non-null  category      
 7   Product         199177 non-null  category      
 8   Product ID      199177 non-null  category      
 9   Payment Method  71311 non-null   category      
 10  Revenues        199177 non-null  float32       
 11  Products        199177 non-null  int8          
dtypes: category(9), datetime64[ns](1), float32(1), int8(1)
memory usage: 6.9 MB


In [209]:
# renomeando a coluna products para evitar ambiguidades
df = df.rename(columns={'Products' : 'Quantity'})
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199177 entries, 0 to 199176
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            199177 non-null  datetime64[ns]
 1   Store ID        199177 non-null  category      
 2   Category        199177 non-null  category      
 3   Sale ID         199177 non-null  category      
 4   Status          199177 non-null  category      
 5   Payment Type    198324 non-null  category      
 6   Place or Sale   199177 non-null  category      
 7   Product         199177 non-null  category      
 8   Product ID      199177 non-null  category      
 9   Payment Method  71311 non-null   category      
 10  Revenues        199177 non-null  float32       
 11  Quantity        199177 non-null  int8          
dtypes: category(9), datetime64[ns](1), float32(1), int8(1)
memory usage: 6.9 MB


## Investigando o estado geral dos dados

In [210]:
# investigando nulos
df.isna().any()

Date              False
Store ID          False
Category          False
Sale ID           False
Status            False
Payment Type       True
Place or Sale     False
Product           False
Product ID        False
Payment Method     True
Revenues          False
Quantity          False
dtype: bool

In [211]:
# investigando linhas nulas usando filtros
masknotNanPaymentType = df['Payment Type'].notnull() # filtro não nulos em payment type
masknotNanPaymentMethod = df['Payment Method'].notnull() # filtro não nulos em payment method
nulosPaymentType = df[~masknotNanPaymentType]
nulosPaymentType

,Date,Store ID,Category,Sale ID,Status,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity
167387,2022-05-02 17:09:13,138,Casual Line,3394022,Delivered,NaN,Smartphone,Bags,1TB,NaN,55.00,1
168208,2022-04-29 10:41:09,138,Hot - Purist,3354674,Delivered,NaN,Tablet,Double espresso,1PB,NaN,5.20,1
168209,2022-04-29 10:41:09,138,Personalization,3354674,Delivered,NaN,Tablet,Without Lid,9PE,NaN,0.00,1
168210,2022-04-29 10:41:09,138,Hot - Purist,3354674,Delivered,NaN,Tablet,Coffee latte,4TW,NaN,8.90,1
168211,2022-04-29 10:41:09,138,Personalization,3354674,Delivered,NaN,Tablet,Lungo (double espresso),3PE,NaN,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
174511,2022-04-07 10:06:38,138,Personalization,3112708,Delivered,NaN,Tablet,Without Lid,9PE,NaN,0.00,1
174512,2022-04-07 10:06:38,138,Hot - Purist,3112708,Delivered,NaN,Tablet,Americano,2AM,NaN,6.80,1
188431,2022-02-17 10:25:57,138,Cold - Flavored,2631175,Delivered,NaN,Smartphone,Ginger,3BG,NaN,10.72,1
188432,2022-02-17 10:25:57,138,Personalization,2631175,Delivered,NaN,Smartphone,With lid 470ml,CTP1,NaN,0.00,1


In [212]:
nulosPaymentMethod = df[~masknotNanPaymentMethod]
nulosPaymentMethod

,Date,Store ID,Category,Sale ID,Status,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity
64835,2023-06-01 14:09:48,138,Personalization,8186487,Delivered,Tablet,Tablet,Average Vanilla (10ml),NRVL,NaN,0.00,1
64836,2023-06-01 14:09:48,138,Milkshakes,8186487,Delivered,Tablet,Tablet,Mocha milkshake,SHMO,NaN,21.00,1
64837,2023-06-01 14:09:48,138,Personalization,8186487,Delivered,Tablet,Tablet,With Lid 250ml,CTP2,NaN,0.00,1
64838,2023-06-01 14:09:48,138,Ice Cream,8186487,Delivered,Tablet,Tablet,Extra ice cream,ICEAFF,NaN,21.00,1
64839,2023-06-01 14:09:48,138,Hot - Flavored,8186487,Delivered,Tablet,Tablet,Vanilla latte,2VLT,NaN,13.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...
199172,2022-01-02 11:20:58,138,Sweets,2265118,Delivered,APP,Smartphone,Cake,8BOLO8,NaN,11.74,2
199173,2022-01-02 11:20:58,138,Personalization,2265118,Delivered,APP,Smartphone,Lactose free,1PE,NaN,1.45,1
199174,2022-01-02 11:20:58,138,Personalization,2265118,Delivered,APP,Smartphone,Average Sugar,NRSU,NaN,0.00,1
199175,2022-01-02 11:20:58,138,Cold - Purist,2265118,Delivered,APP,Smartphone,Green tea latte (iced),4MIL,NaN,10.46,1


In [213]:
# filtrando cada coluna a procura dos valores únicos que se repetem
paymentType = df['Payment Type'].unique()
paymentType = pd.DataFrame(paymentType)
status = df['Status'].unique()
status = pd.DataFrame(status)
paymentMethod = df['Payment Method'].unique()
paymentMethod = pd.DataFrame(paymentMethod)
placeOrSale = df['Place or Sale'].unique()
placeOrSale = pd.DataFrame(placeOrSale)

In [214]:
paymentType

,0
0,Tablet
1,APP
2,Contingency
3,NaN


In [215]:
status

,0
0,Delivered


In [216]:
paymentMethod

,0
0,Tablet - Card
1,Tablet - Pix
2,APP - Wallet
3,APP - Card
4,APP - PIX
5,Contingency - Ifood
6,Contingency - Money
7,Contingency - Card
8,Contingency - PIX
9,NaN


In [217]:
placeOrSale

,0
0,Tablet
1,Smartphone


## Separando linhas com receita zerada e eliminando colunas desnecessárias

In [218]:
maskZeroRevenue = df['Revenues'] == 0
dfZeroRevenue = df[maskZeroRevenue]
df = df[~maskZeroRevenue]
df['Revenues'] = df['Revenues'].round(decimals=2)
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 93392 entries, 2 to 199175
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            93392 non-null  datetime64[ns]
 1   Store ID        93392 non-null  category      
 2   Category        93392 non-null  category      
 3   Sale ID         93392 non-null  category      
 4   Status          93392 non-null  category      
 5   Payment Type    93042 non-null  category      
 6   Place or Sale   93392 non-null  category      
 7   Product         93392 non-null  category      
 8   Product ID      93392 non-null  category      
 9   Payment Method  32321 non-null  category      
 10  Revenues        93392 non-null  float32       
 11  Quantity        93392 non-null  int8          
dtypes: category(9), datetime64[ns](1), float32(1), int8(1)
memory usage: 4.9 MB


In [219]:
# a coluna de status não serve pra coisa alguma, vamos eliminá-la:
df = df.drop(columns='Status')
df

,Date,Store ID,Category,Sale ID,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity
2,2023-12-31 17:16:23,138,Hot - Flavored,10924941,Tablet,Tablet,Caramel latte,1FC,Tablet - Card,14.500000,1
5,2023-12-31 17:13:38,138,Methods,10924929,Tablet,Tablet,V60,1HDV60,Tablet - Pix,19.799999,2
11,2023-12-31 17:01:47,138,Hot - Purist,10924879,APP,Smartphone,Double espresso,1PB,APP - Wallet,6.830000,1
13,2023-12-31 16:53:10,138,Cold - Purist,10924839,Tablet,Tablet,Iced coffee latte,5IL,Tablet - Card,12.900000,1
19,2023-12-31 16:39:10,138,Hot - Flavored,10924770,Tablet,Tablet,Mocha latte,3MC,Tablet - Pix,13.100000,1
...,...,...,...,...,...,...,...,...,...,...,...
199168,2022-01-02 11:20:58,138,Personalization,2265118,APP,Smartphone,Oatmilk,LVG,NaN,4.680000,1
199171,2022-01-02 11:20:58,138,Hot - Flavored,2265118,APP,Smartphone,Mocha latte,3MC,NaN,8.590000,1
199172,2022-01-02 11:20:58,138,Sweets,2265118,APP,Smartphone,Cake,8BOLO8,NaN,11.740000,2
199173,2022-01-02 11:20:58,138,Personalization,2265118,APP,Smartphone,Lactose free,1PE,NaN,1.450000,1


In [220]:
# reanalisando os nulos em payment type e payment method
PtypezeroRevenue = nulosPaymentType['Revenues'] == 0
nulosPaymentType = nulosPaymentType[~PtypezeroRevenue]
nulosPaymentType


,Date,Store ID,Category,Sale ID,Status,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity
167387,2022-05-02 17:09:13,138,Casual Line,3394022,Delivered,NaN,Smartphone,Bags,1TB,NaN,55.00,1
168208,2022-04-29 10:41:09,138,Hot - Purist,3354674,Delivered,NaN,Tablet,Double espresso,1PB,NaN,5.20,1
168210,2022-04-29 10:41:09,138,Hot - Purist,3354674,Delivered,NaN,Tablet,Coffee latte,4TW,NaN,8.90,1
173526,2022-04-09 20:33:24,138,Cold - Flavored,3146521,Delivered,NaN,Tablet,Salted caramel latte (iced),2SC,NaN,14.10,1
173529,2022-04-09 20:30:25,138,Cold - Flavored,3146513,Delivered,NaN,Tablet,Ginger,3BG,NaN,13.40,1
...,...,...,...,...,...,...,...,...,...,...,...,...
174505,2022-04-07 11:31:22,138,Cold - Flavored,3113883,Delivered,NaN,Tablet,Salted caramel latte (iced),2SC,NaN,14.10,1
174507,2022-04-07 11:20:28,138,Hot - Purist,3113766,Delivered,NaN,Tablet,Double espresso,1PB,NaN,5.20,1
174509,2022-04-07 10:09:46,138,Hot - Flavored,3112757,Delivered,NaN,Tablet,Vanilla latte,2VLT,NaN,10.70,1
174512,2022-04-07 10:06:38,138,Hot - Purist,3112708,Delivered,NaN,Tablet,Americano,2AM,NaN,6.80,1


In [221]:
PMethodZeroRevenue = nulosPaymentMethod['Revenues'] == 0
nulosPaymentMethod = nulosPaymentMethod[~PMethodZeroRevenue]
nulosPaymentMethod

,Date,Store ID,Category,Sale ID,Status,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity
64836,2023-06-01 14:09:48,138,Milkshakes,8186487,Delivered,Tablet,Tablet,Mocha milkshake,SHMO,NaN,21.00,1
64838,2023-06-01 14:09:48,138,Ice Cream,8186487,Delivered,Tablet,Tablet,Extra ice cream,ICEAFF,NaN,21.00,1
64839,2023-06-01 14:09:48,138,Hot - Flavored,8186487,Delivered,Tablet,Tablet,Vanilla latte,2VLT,NaN,13.80,1
64841,2023-06-01 14:03:37,138,Personalization,8186351,Delivered,Tablet,Tablet,Banana Cake,BBAN,NaN,1.90,1
64844,2023-06-01 14:03:37,138,Sweets,8186351,Delivered,Tablet,Tablet,Cake,8BOLO8,NaN,7.10,1
...,...,...,...,...,...,...,...,...,...,...,...,...
199168,2022-01-02 11:20:58,138,Personalization,2265118,Delivered,APP,Smartphone,Oatmilk,LVG,NaN,4.68,1
199171,2022-01-02 11:20:58,138,Hot - Flavored,2265118,Delivered,APP,Smartphone,Mocha latte,3MC,NaN,8.59,1
199172,2022-01-02 11:20:58,138,Sweets,2265118,Delivered,APP,Smartphone,Cake,8BOLO8,NaN,11.74,2
199173,2022-01-02 11:20:58,138,Personalization,2265118,Delivered,APP,Smartphone,Lactose free,1PE,NaN,1.45,1


## Verificando a relação entre Revenues e Quantity

In [222]:
# agrupando linhas dentro do ticket
tickets = df.groupby('Sale ID', observed=False).aggregate({'Revenues': 'sum', 'Product': 'nunique', 'Quantity': 'sum'})
tickets['Revenues'] = tickets['Revenues'].astype(float)
tickets['Revenues'] = tickets['Revenues'].round(decimals=2)
tickets

,Revenues,Product,Quantity
Sale ID,,,
2265118,36.92,5,6
2265312,20.80,2,2
2265402,15.70,2,2
2265407,24.40,3,3
2265475,5.20,1,1
...,...,...,...
10924770,13.10,1,1
10924839,12.90,1,1
10924879,6.83,1,1


In [223]:
# usando como exemplo o ticket 2265118:
mask2265118 = df['Sale ID'] == 2265118
df[mask2265118]

,Date,Store ID,Category,Sale ID,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity
199168,2022-01-02 11:20:58,138,Personalization,2265118,APP,Smartphone,Oatmilk,LVG,NaN,4.68,1
199171,2022-01-02 11:20:58,138,Hot - Flavored,2265118,APP,Smartphone,Mocha latte,3MC,NaN,8.59,1
199172,2022-01-02 11:20:58,138,Sweets,2265118,APP,Smartphone,Cake,8BOLO8,NaN,11.74,2
199173,2022-01-02 11:20:58,138,Personalization,2265118,APP,Smartphone,Lactose free,1PE,NaN,1.45,1
199175,2022-01-02 11:20:58,138,Cold - Purist,2265118,APP,Smartphone,Green tea latte (iced),4MIL,NaN,10.46,1


## Criando colunas para agregação de data e hora

In [224]:
#criando colunas de dia, mês, ano e hora
df.loc[::,'year'] = df.loc[::,'Date'].dt.year.astype('category')
df.loc[::,'month_of_year'] = df.loc[::,'Date'].dt.month.astype('category')
df.loc[::,'week_of_year'] = df.loc[::,'Date'].dt.isocalendar().week.astype('category')
df.loc[::,'weekday'] = df.loc[::,'Date'].dt.weekday.astype('category')
df.loc[::,'day'] = df.loc[::,'Date'].dt.date.astype('category')
df.loc[::,'day_of_year'] = df.loc[::,'Date'].dt.dayofyear.astype('category')
df.loc[::,'hour'] = df.loc[::,'Date'].dt.hour.astype('category')
df

,Date,Store ID,Category,Sale ID,Payment Type,Place or Sale,Product,Product ID,Payment Method,Revenues,Quantity,year,month_of_year,week_of_year,weekday,day,day_of_year,hour
2,2023-12-31 17:16:23,138,Hot - Flavored,10924941,Tablet,Tablet,Caramel latte,1FC,Tablet - Card,14.500000,1,2023,12,52,6,2023-12-31,365,17
5,2023-12-31 17:13:38,138,Methods,10924929,Tablet,Tablet,V60,1HDV60,Tablet - Pix,19.799999,2,2023,12,52,6,2023-12-31,365,17
11,2023-12-31 17:01:47,138,Hot - Purist,10924879,APP,Smartphone,Double espresso,1PB,APP - Wallet,6.830000,1,2023,12,52,6,2023-12-31,365,17
13,2023-12-31 16:53:10,138,Cold - Purist,10924839,Tablet,Tablet,Iced coffee latte,5IL,Tablet - Card,12.900000,1,2023,12,52,6,2023-12-31,365,16
19,2023-12-31 16:39:10,138,Hot - Flavored,10924770,Tablet,Tablet,Mocha latte,3MC,Tablet - Pix,13.100000,1,2023,12,52,6,2023-12-31,365,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199168,2022-01-02 11:20:58,138,Personalization,2265118,APP,Smartphone,Oatmilk,LVG,NaN,4.680000,1,2022,1,52,6,2022-01-02,2,11
199171,2022-01-02 11:20:58,138,Hot - Flavored,2265118,APP,Smartphone,Mocha latte,3MC,NaN,8.590000,1,2022,1,52,6,2022-01-02,2,11
199172,2022-01-02 11:20:58,138,Sweets,2265118,APP,Smartphone,Cake,8BOLO8,NaN,11.740000,2,2022,1,52,6,2022-01-02,2,11
199173,2022-01-02 11:20:58,138,Personalization,2265118,APP,Smartphone,Lactose free,1PE,NaN,1.450000,1,2022,1,52,6,2022-01-02,2,11


In [225]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 93392 entries, 2 to 199175
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            93392 non-null  datetime64[ns]
 1   Store ID        93392 non-null  category      
 2   Category        93392 non-null  category      
 3   Sale ID         93392 non-null  category      
 4   Payment Type    93042 non-null  category      
 5   Place or Sale   93392 non-null  category      
 6   Product         93392 non-null  category      
 7   Product ID      93392 non-null  category      
 8   Payment Method  32321 non-null  category      
 9   Revenues        93392 non-null  float32       
 10  Quantity        93392 non-null  int8          
 11  year            93392 non-null  category      
 12  month_of_year   93392 non-null  category      
 13  week_of_year    93392 non-null  category      
 14  weekday         93392 non-null  category      
 15  day   